In [53]:
import os
import pandas as pd
import geopandas as gpd
import fiona
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sqlalchemy import text
from django.contrib.gis.geos import MultiLineString, LineString

### Import TXT file and manage data - CTA Bus Stops

In [54]:
# Import txt file of CTA bus stops
column_names = [
    "route_id",
    "var_1",
    "stop_name",
    "stop_id",
    "latitude",
    "longitude",
    "var_2",
    "var_3",
]
df_stops = pd.read_csv("../data/CTA_STOP_XFERS.txt", header=None, names=column_names)

# Remove unnecessary fields and add "type" field
df_stops = df_stops[["stop_id", "route_id", "stop_name", "latitude", "longitude"]]
df_stops["type"] = "CTA"

# Filter buses that don't exist (no available in routes file) and train (focus on buses)
routes_to_filter = ['992', '999', 'Blue', 'Brown', 'Green', 'Orange', 'Pink', 
                    'Purple', 'Red', 'Yellow']

df_stops = df_stops[~df_stops['route_id'].astype(str).isin(routes_to_filter)]

df_stops

,stop_id,route_id,stop_name,latitude,longitude,type
0,17298,152,Addison & Halsted,41.947298,-87.649178,CTA
1,12513,152,Addison & Hamlin,41.946408,-87.722580,CTA
2,17358,152,Addison & Harlem,41.945240,-87.806693,CTA
3,12530,152,Addison & Hoyne,41.946782,-87.681080,CTA
4,12510,152,Addison & Karlov,41.946313,-87.729287,CTA
...,...,...,...,...,...,...
14664,4164,X9,Ashland Orange Line Station,41.839099,-87.665390,CTA
14665,6179,X9,Ashland/63rd Street (Green Line),41.778936,-87.664081,CTA
14666,5668,X9,Irving Park & Clark,41.954310,-87.662413,CTA
14667,15930,X9,Irving Park & Fremont,41.954603,-87.651532,CTA


### Import GeoJSON and manage data - Shuttle Stops

In [55]:
# Open GeoJSON
shuttle_data = gpd.read_file("../data/uchicago_shuttle_stops.geojson")

# Generate dataframe to add to current dataframe of stops
shuttle_df = pd.DataFrame({
    "stop_id": (shuttle_data["stop_id"].astype(int) * 10**3).astype(str), # assure unique stop id
    "route_id": shuttle_data["route_name"],
    "stop_name": shuttle_data["stop_name"], 
    "latitude": shuttle_data.geometry.y,
    "longitude": shuttle_data.geometry.x,
    "type": "SHUTTLE"
})

# Unique routes to check information
print(shuttle_df["route_id"].unique())

# Replace "Downtown Campus Connector" to "Downtown Connector" to keep len of id
shuttle_df["route_id"] = shuttle_df["route_id"].replace("Downtown Campus Connector", 
                                                        "Downtown Connector")

# No information on UChicago bus webpage related to "GE Gleacher Express" and
# "MED Medical Shuttle" (also has only 2 and 1 bus stops respectively), delete it from the data
# Link: https://safety-security.uchicago.edu/en/transportation
shuttle_df = shuttle_df[
    ~shuttle_df["route_id"].isin(["GE Gleacher Express", "MED Medical Shuttle"])
]

# Keep data only for Daytime Shuttle Routes
# Link: https://safety-security.uchicago.edu/en/transportation
shuttle_df = shuttle_df[
    shuttle_df["route_id"].isin([
        "53rd Street Express", 
        "Apostolic",
        "Apostolic/Drexel",
        "Downtown Connector",
        "Drexel",
        "Friend Center/Metra",
        "Red Line/Arts Block",
        "Midway Metra",
    ])
]

# Add shuttle data to dataframe of stops
df_stops = pd.concat([df_stops, shuttle_df], ignore_index=True)
df_stops

['Apostolic' 'South' 'Downtown Campus Connector' 'GE Gleacher Express'
 'Red Line/Arts Block' 'Regents Express' 'Midway Metra'
 'South Loop Shuttle' 'Central' 'East' 'MED Medical Shuttle' 'North'
 '53rd Street Express' 'Apostolic/Drexel' 'Drexel' 'Friend Center/Metra']


,stop_id,route_id,stop_name,latitude,longitude,type
0,17298,152,Addison & Halsted,41.947298,-87.649178,CTA
1,12513,152,Addison & Hamlin,41.946408,-87.722580,CTA
2,17358,152,Addison & Harlem,41.945240,-87.806693,CTA
3,12530,152,Addison & Hoyne,41.946782,-87.681080,CTA
4,12510,152,Addison & Karlov,41.946313,-87.729287,CTA
...,...,...,...,...,...,...
14574,8618000,Friend Center/Metra,Ronald McDonald House & Friend Center,41.795381,-87.604187,SHUTTLE
14575,140009000,Friend Center/Metra,55th Street & University,41.795011,-87.597999,SHUTTLE
14576,8645000,Friend Center/Metra,Hyde Park Shopping Center (S),41.795096,-87.587745,SHUTTLE
14577,8619000,Friend Center/Metra,57th Street Metra Station (E),41.791490,-87.586864,SHUTTLE


### Import csv file and manage data - CTA Bus Routes

In [56]:
# Import file
column_names = [
    "geometry",
    "route_id",
    "route_name",
    "weekday",
    "saturday",
    "sunday",
]
df_routes = pd.read_csv("../data/CTA_-_Bus_Routes_20250514.csv")
df_routes.columns = column_names

# Remove unnecessary fields and add necessary fields
df_routes = df_routes[["route_id", "route_name", "geometry"]]
df_routes["type"] = "CTA"
df_routes


,route_id,route_name,geometry,type
0,12,ROOSEVELT,MULTILINESTRING ((-87.64790999981973 41.867129...,CTA
1,121,UNION/STREETERVILLE EXPRESS,MULTILINESTRING ((-87.62449999918586 41.888329...,CTA
2,1,BRONZEVILLE/UNION STATION,MULTILINESTRING ((-87.62325999972036 41.831030...,CTA
3,108,HALSTED/95TH,MULTILINESTRING ((-87.59052999958188 41.655770...,CTA
4,11,LINCOLN,MULTILINESTRING ((-87.68882999971883 41.966760...,CTA
...,...,...,...,...
122,120,OGILVIE/STREETERVILLE EXPRESS,MULTILINESTRING ((-87.62449999918586 41.888329...,CTA
123,126,JACKSON,MULTILINESTRING ((-87.77390000023009 41.876470...,CTA
124,20,MADISON,MULTILINESTRING ((-87.77390694614428 41.880080...,CTA
125,128,SOLDIER FIELD EXPRESS,MULTILINESTRING ((-87.61698999997792 41.865019...,CTA


### Import shapefiles and manage data - Shuttle Routes

In [57]:
def multilinestring_from_shapefile(route_name):
    """
    Create a MultiLineString WKT from a zip file that contains a shapefile for a shuttle bus.
    """
    lines = []

    # Construct the ZIP file path and shapefile path within the ZIP for fiona
    zip_path = f"../data/shuttle_routes.zip"
    shapefile_path = f"zip://{zip_path}!{route_name}.shp"
    
    # Open file with fione
    ## NOTE: I saw this blog to see how to open shapefiles with fiona
    ## Link: https://kartoza.com/blog/python/reading-and-writing-shapefiles-in-python-with-fiona
    with fiona.open(shapefile_path) as shapefile:
        for feature in shapefile:
            # Get coordinates of the shapefile
            coordinates = feature["geometry"]["coordinates"]
            # Create LineString from coordinates and append to lines list
            line = LineString(coordinates)
            lines.append(line)
    
    # Create MultiLineString from list of LineStrings
    multilinestring = MultiLineString(lines)
    
    return multilinestring.wkt

# Generate list with file names and id of each shuttle
shuttle_route_file_name = [
    ("53rd_express", "53rd Street Express"),
    ("apostolic", "Apostolic"),
    ("apostolic_drexel", "Apostolic/Drexel"),
    ("downtown_conector", "Downtown Connector"),
    ("drexel_route", "Drexel"), 
    ("friend", "Friend Center/Metra"),
    ("red", "Red Line/Arts Block"), 
    ("midway_metra", "Midway Metra"),
]

# Generate row for each shuttle route and add it to routes dataframe
for shuttle_file_name, shuttle_id in shuttle_route_file_name:
    new_row = pd.DataFrame({
        "route_id": shuttle_id,
        "route_name": shuttle_id.upper(),
        "geometry": [multilinestring_from_shapefile(shuttle_file_name)],
        "type": ["SHUTTLE"]
    })
    df_routes = pd.concat([df_routes, new_row], ignore_index=True)

df_routes

,route_id,route_name,geometry,type
0,12,ROOSEVELT,MULTILINESTRING ((-87.64790999981973 41.867129...,CTA
1,121,UNION/STREETERVILLE EXPRESS,MULTILINESTRING ((-87.62449999918586 41.888329...,CTA
2,1,BRONZEVILLE/UNION STATION,MULTILINESTRING ((-87.62325999972036 41.831030...,CTA
3,108,HALSTED/95TH,MULTILINESTRING ((-87.59052999958188 41.655770...,CTA
4,11,LINCOLN,MULTILINESTRING ((-87.68882999971883 41.966760...,CTA
...,...,...,...,...
130,Downtown Connector,DOWNTOWN CONNECTOR,MULTILINESTRING ((-87.60607322727228 41.787700...,SHUTTLE
131,Drexel,DREXEL,MULTILINESTRING ((-87.60610683934425 41.791319...,SHUTTLE
132,Friend Center/Metra,FRIEND CENTER/METRA,MULTILINESTRING ((-87.60485677867393 41.787698...,SHUTTLE
133,Red Line/Arts Block,RED LINE/ARTS BLOCK,MULTILINESTRING ((-87.60117628798767 41.785914...,SHUTTLE


### Ingest data into SQL

In [58]:
# Load .env
load_dotenv("../.env")
db_url = os.getenv("DATABASE_URL")

# Create SQLAlchemy engine
engine = create_engine(db_url)

# Delete all observations
with engine.connect() as connection:
    connection.execute(text("DELETE FROM apt_app_stops;"))
    connection.execute(text("DELETE FROM apt_app_routes;"))
    connection.commit()


# Ingest data to corresponding table
df_stops.to_sql("apt_app_stops", engine, if_exists="replace", index=False)
df_routes.to_sql("apt_app_routes", engine, if_exists="replace", index=False)

135

### Check SQL data - Stops

In [59]:
# Count how many rows has the SQL table
db_count = pd.read_sql("SELECT COUNT(*) AS count FROM apt_app_stops", engine).iloc[0, 0]
print("Total rows:", db_count)

# head
df_preview_stops = pd.read_sql("SELECT * FROM apt_app_stops LIMIT 5", engine)
print(df_preview_stops)

Total rows: 14579
  stop_id route_id          stop_name   latitude  longitude type
0   17298      152  Addison & Halsted  41.947298 -87.649178  CTA
1   12513      152   Addison & Hamlin  41.946408 -87.722580  CTA
2   17358      152   Addison & Harlem  41.945240 -87.806693  CTA
3   12530      152    Addison & Hoyne  41.946782 -87.681080  CTA
4   12510      152   Addison & Karlov  41.946313 -87.729287  CTA


### Check SQL data - Routes

In [60]:
# Count how many rows has the SQL table
db_count = pd.read_sql("SELECT COUNT(*) AS count FROM apt_app_routes", engine).iloc[0, 0]
print("Total rows:", db_count)

# head
df_preview_stops = pd.read_sql("SELECT * FROM apt_app_routes LIMIT 5", engine)
print(df_preview_stops)

Total rows: 135
  route_id                   route_name  \
0       12                    ROOSEVELT   
1      121  UNION/STREETERVILLE EXPRESS   
2        1    BRONZEVILLE/UNION STATION   
3      108                 HALSTED/95TH   
4       11                      LINCOLN   

                                            geometry type  
0  MULTILINESTRING ((-87.64790999981973 41.867129...  CTA  
1  MULTILINESTRING ((-87.62449999918586 41.888329...  CTA  
2  MULTILINESTRING ((-87.62325999972036 41.831030...  CTA  
3  MULTILINESTRING ((-87.59052999958188 41.655770...  CTA  
4  MULTILINESTRING ((-87.68882999971883 41.966760...  CTA  
